In [33]:
%%pyspark

from azureml.opendatasets import NycTlcYellow

from datetime import datetime
import calendar

account_name = "<Storage アカウント名>"
container_name = "<Storage コンテナ名>"
account_key = "<Storage アカウントキー>"
spark.conf.set("fs.azure.account.key.%s.blob.core.windows.net" %account_name ,account_key)

for i in range(1, 7):
    dt = datetime(2019, i, 1)
    filepath = "wasbs://%s@%s.blob.core.windows.net/source/nyctaxidata/year=%s/month=%s/" %(container_name, account_name, dt.strftime("%Y"), dt.strftime("%m"))
    start_date = dt.replace(day=1)
    end_date = dt.replace(day=calendar.monthrange(dt.year, dt.month)[1])
    
    nyc_tlc = NycTlcYellow(start_date=start_date, end_date=end_date)
    nyc_tlc_df = nyc_tlc.to_spark_dataframe()
    
    nyc_tlc_df.coalesce(1).write.mode("overwrite").option("header", "True").csv(filepath)

    files = mssparkutils.fs.ls(filepath)
    for file in files:
        if file.name.endswith(".csv"):
            mssparkutils.fs.mv(filepath + file.name, filepath + dt.strftime("%Y_%m") + "_NycTlcYellow.csv")